In [19]:
import pandas as pd
import numpy as np
import re


Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so we should get rid of them, and we should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [ ]:
def main():
    df = pd.read_excel('assets/Energy Indicators.xls', header=17, skipfooter=38, sheet_name='Energy')
    df.drop(columns=[df.columns[0], df.columns[1]], axis=1, inplace=True)
    df.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    df.set_index("Country", inplace=True)
    for i in range(len(df['Energy Supply'])):
        if df['Energy Supply'][i] == '...':
            df['Energy Supply'][i] = np.NaN
    df['Energy Supply'] = df['Energy Supply'].apply(lambda x: x*1_000_000)
    df.rename(index={"Republic of Korea": "South Korea", "United States of America20": "United States",
                     "United Kingdom of Great Britain and Northern Ireland19": "United Kingdom",
                     "China, Hong Kong Special Administrative Region3": "Hong Kong"}, inplace = True)
    pattern = r"[A-Z]\w+ \w+"
    for index in df.index:
        df.rename(index={f"{index}": re.sub('\d+', '', index)}, inplace = True)
        df.rename(index={f"{index}": re.sub(' [(].+[)]', '', index)}, inplace = True)
    df = df.reset_index()
    df.rename(columns={'Country': 'Country Name'}, inplace=True)
    
    gdp = pd.read_csv('assets/world_bank.csv', header=4)
    gdp.set_index("Country Name", inplace=True)
    gdp.rename(index={"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}, inplace=True)
    gdp.drop(columns=gdp.columns[0:-10], inplace=True)
    gdp = gdp.reset_index()
    
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    ScimEn.set_index('Rank', inplace=True)    
    ScimEn.rename(columns={'Country': 'Country Name'}, inplace=True)
    ScimEn = ScimEn.reset_index()
    
    result1 = pd.merge(ScimEn, df, how='inner', on='Country Name')
    result = pd.merge(result1, gdp, how='inner', on='Country Name')
    test = pd.merge(ScimEn, gdp, how='inner', on='Country Name')
    result.set_index('Country Name', inplace=True)
    result = pd.DataFrame(result[:15])
    meanSupplyPerCapita = result['Energy Supply per Capita'].mean(axis=0)
    ReNew = result.sort_values(by=['% Renewable'], ascending=False)
    return result
    raise NotImplementedError()
main()